In [16]:
import math
from decimal import *
import os
import re
import shutil
import sys
import warnings

import chardet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from statistics import median

from constants import bins_axis_names, bin_dimensions_regex, boat_speed_feature, \
                    boxplot_axis_name, feature_regex, identifier_features, \
                    fiber_optics_structure_features, fiber_optics_appendix_features, \
                    other_sensor_features, statistics_features, wind_features

In [10]:
def read_csv(fnames):
    """Read CSV file(s) to a Pandas Dataframe"""
    def detect_and_read(fname):
        with open(fname, 'rb') as f:
            return pd.read_csv(fname, sep=';', encoding=chardet.detect(f.read())['encoding'])
    df = pd.concat(map(lambda fname: detect_and_read(fname), fnames), sort=False)
    return df

In [11]:
# Ignore RunTimeWarning.
warnings.filterwarnings('ignore')

# Ignore PyPlot warning.
plt.rcParams.update({'figure.max_open_warning': 0})

# Version number.
version = 'v2'

In [42]:
#Prend en argument un csv et renvoie un dataframe avec uniquement les données utilisables pour analyser le comportement du bateau
def clean_data(data, min_boat_speed):
    
    df = read_csv(map(lambda fname: data.format(version, fname), ['data-part-1', 'data-part-2']))
    
    df = df.drop_duplicates()
    
    l1 = len(df)
    
    index_to_be_deleted = []
    
    #Indices pour lesquels le bateau est face au vent
    ind1 = df[df["WTP2_TW_angle"] == 0].index
    
    #Indices pour lesquels le bateau est en vent arrière
    ind2 = df[df["WTP2_TW_angle"].isin([-179, -180, 179])].index
    
    #Indices pour lesquels le bateau avance à moins de 5 noeuds
    ind3 = df[df['VarFilter_WTP_SelBoatSpd'] < min_boat_speed].index
    
    index_to_be_deleted.extend(ind3)
    
    for i1 in ind1:
        index_to_be_deleted.extend(np.arange(i1-600, i1+600))

    for i2 in ind2:
        index_to_be_deleted.extend(np.arange(i2-600, i2+600))

    #On supprime les indices qui apparaissent plusieurs fois
    final_list = list(set(index_to_be_deleted))
    
    l2 = len(final_list)
    
    print(Decimal(l2/l1*100).quantize(Decimal('.1'), rounding=ROUND_HALF_UP),'% des données ont été ignorées')
    
    df = df.drop(final_list, errors = 'ignore')
    
    #On réinitialise les indices
    df = df.reset_index(drop = True)
    
    return (df)

In [43]:
df = clean_data('datalog-psm2-181018_0743_1447_1Hz.csv', 5)

38.7 % des données ont été ignorées
